In [111]:
import pandas as pd 
import numpy as np 
import requests
import csv 
import re
from bs4 import BeautifulSoup
from selenium import webdriver

In [112]:
def title_link(s):
    tit=[]
    link=[]
    sta=s.find_all('a',attrs={'class':"bizlinkurl"}) 
    for k in sta:
        tit.append(k.text)
        link.append(k['href'])
    d={
        'title_link':tit,
        'links':link
    }
    df=pd.DataFrame(d)
    return df

In [113]:
def address(s):
    address=[]
    sta=s.find_all('div',attrs={'class':"locality"}) 
    for a in sta:
        address.append(a.text.split()[0])
    return address


 

In [114]:
def location(s):
    location=[]
    sta=s.find_all('a',attrs={'class':"view-on-map"})    
    for a in sta:
        location.append(a['data-latlong'])
    return location

In [115]:
def prices(s):
    prices=[]
    sta=s.find_all('span',attrs={'class':"deposit"})    
    for i in sta:
        prices.append(i.text)
    """    sta=s.find_all('span',attrs={'class':"Deposit"})    
    for i in sta:
        prices.append(i.text)"""
    return prices

In [116]:
def financial(s):
    financial=[]
    sta=s.find_all('span',attrs={'class':"price primary"})    
    for i in sta:
        financial.append(i.text)
    return financial

In [117]:
def amenities(s):
    ammenities=[]
    sta=s.find_all('div',attrs={'class':"amenities flat"})   
    for i in sta:
        ammenities.append(i.text)
    return ammenities

In [118]:
def spec_flat(s):
    spec_flat=[]
    sta=s.find_all('div',attrs={'class':"spec flat"})   
    for i in sta:
        spec_flat.append(i.text)
    return spec_flat

In [119]:
def spec_flat_necce(s):
    spec_flat_necce=[]
    sta=s.find_all('div',attrs={'class':"spec icon primary"})   
    for i in sta:
        spec_flat_necce.append(i.text)
    return spec_flat_necce

In [120]:
def the_area(s):
    clean=[]
    area=[]
    sta=s.find_all('span',attrs={'class':"feature primary"})   
    for i in sta:
        area.append(i.text.split())
    for a in area:
        clean.append(" ".join(a[-3:]))
    return clean

In [121]:
# for a in area:
#     print(" ".join(a[-3:]))

1150 Sq Feet
1050 Sq Feet
250 Sq Feet
300 Sq Feet
930 Sq Feet
625 Sq Feet
400 Sq Feet
600 Sq Feet
860 Sq Feet
680 Sq Feet
621 Sq Feet
285 Sq Feet
585 Sq Feet
400 Sq Feet
675 Sq Feet
Availability Status: Immediate
860 Sq Feet
285 Sq Feet
585 Sq Feet
400 Sq Feet
675 Sq Feet
400 Sq Feet
200 Sq Feet
1509 Sq Feet
900 Sq Feet


In [122]:
def dates_bro(s):
    dates_bro=[]
    sta=s.find_all('div',attrs={'class':"posted"})   
    for i in sta:
        dates_bro.append(i.text)
    return dates_bro

In [123]:
def the_text(s):
    the_text=[]
    sta=s.find_all('span',attrs={'class':"listing-desc view-less"})   
    for i in sta:
        the_text.append(i.text)

In [124]:
url="https://property.sulekha.com/apartments-flats-for-rent/pune#"
j=requests.get(url)
k=j.text
s=BeautifulSoup(k,'lxml')

In [125]:
def the_filter(df):
    the_liost=df['financial']
    prices=[]
    deposit=[]
    for l in the_liost:
        k=l.split(' ')
        prices.append(k[3])
        deposit.append(k[-4])
    df['prices']=prices
    df['deposit']=deposit
    k=df['dates_bro']
    the_posted_dates=[]
    details_of=[]
    for i in k :
        i=i.strip()
        i=i.split(" ")
        the_posted_dates.append(i[-3:])
        details_of.append(' '.join(i[0:-3]))
    df['the_posted_dates']=the_posted_dates
    df['details']=details_of
    df_new=df.drop(['financial',	'dates_bro'], axis = 1)
    df_new=df_new.drop(['amenities'],axis=1)
    k=len(df['the_posted_dates'])
    the_date=[]
    for i in range(0,k):
        j=df['the_posted_dates'][i]
        the_date.append(" ".join(j))
    df_new['the_posted_dates']=pd.to_datetime(the_date)
    return df_new

In [126]:
def data_frames(s):
    df=title_link(s)
    df['adress']=address(s)
    #df['prices']=prices(s)
    df['financial']=financial(s)
    df['dates_bro']=dates_bro(s)
    df['spec_flat']=spec_flat(s)
    df['spec_flat_necce']=spec_flat_necce(s)
    df['amenities']=amenities(s)
    df['location']=location(s)
    df['area']=the_area(s)
    df=the_filter(df)
    return df

In [127]:
import time 

In [128]:
from selenium import webdriver
from selenium.webdriver.common.keys import Keys
from time import sleep
the_data_pull=[]
# browser = webdriver.Firefox(executable_path=r"C:\Users\swaraj\Desktop\codes\modules\geckodriver-v0.29.1-win64\geckodriver")
browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
browser.get(url)
for i in range(2,7):
    i=str(i)
    browser.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    time.sleep(2)
    try :
        element = browser.find_element_by_link_text("View More")
        k=element.click()
    except:
        pass
    
    #j=requests.get(browser.page_source)
    #k=j.text
    s=BeautifulSoup(browser.page_source,'lxml')
    THE_DA=data_frames(s)
    the_data_pull.append(THE_DA)

    sleep(2)
browser.quit()

In [129]:
df=pd.concat(the_data_pull)

In [130]:
df['links']=df['links'].apply(lambda x:'https://property.sulekha.com'+x )

In [131]:
df

,title_link,links,adress,spec_flat,spec_flat_necce,location,area,prices,deposit,the_posted_dates,details
0,2 BHK Residential Apartment for Rent at kumar ...,https://property.sulekha.com/apartments-flats-...,"Hadapsar,",1 Car Parking 4th (of 11 Floors) UnFurnished F...,Corporation water Drainage Facility 24*7 Power...,"18.515806,73.927164",1150 Sq Feet,for,"45,000",2021-03-01,Building: kumar paradise Broker: Sejal Prop...
1,2 BHK Residential Apartment for Rent at La Glo...,https://property.sulekha.com/apartments-flats-...,Kalyani,2 Car Parking 6th (of 7 Floors) Fully Furnishe...,Drainage Facility Lifts Security +22 Amenities,"0,0",1050 Sq Feet,26,"80,000",2020-12-11,Building: La Gloriosa Broker: Mountain Gate...
2,1 RK Residential Apartment for Rent in Baner Gaon,https://property.sulekha.com/apartments-flats-...,Baner,1 Car Parking 2nd (of 3 Floors) Semi Furnished...,Feng Shui / Vastu,"0,0",250 Sq Feet,"7,000","20,000",2021-03-08,Broker: Life Real Estates
3,1 RK Residential Apartment for Rent in Pavana ...,https://property.sulekha.com/apartments-flats-...,Pavana,No Car Parking 1st (of 1 Floors) UnFurnished B...,Corporation water,"18.6259403,73.7804315",300 Sq Feet,"6,500","15,000",2021-03-10,Broker: Krishna estate agency
4,2 BHK Residential Apartment for Rent in Magarp...,https://property.sulekha.com/apartments-flats-...,"Magarpatta,",1 Car Parking 5th (of 11 Floors) UnFurnished F...,Lifts Park Club House +2 Amenities,"0,0",930 Sq Feet,"18,000","50,000",2021-02-23,Broker: Trimurti Real Estate consultant
...,...,...,...,...,...,...,...,...,...,...,...
20,1 RK Residential Apartment for Rent in Pimpri,https://property.sulekha.com/apartments-flats-...,"Pimpri,",1 Car Parking Ground Floor (of 1 Floors) UnFur...,Corporation water Drainage Facility 24*7 Power...,"18.6297811,73.7997094",400 Sq Feet,"7,000","50,000",2019-06-29,Broker: Vision Associates
21,1 BHK Residential Apartment for Rent in Dhayari,https://property.sulekha.com/apartments-flats-...,Benkar,No Car Parking 3rd (of 4 Floors) UnFurnished F...,Corporation water ATM Visitors Parking +1 Amen...,"0,0",550 Sq Feet,"5,600","20,000",2021-05-15,Broker: Rama Enterprises
22,3 BHK Residential Apartment for Rent at Eterna...,https://property.sulekha.com/apartments-flats-...,"Undri,",1 Car Parking 4th (of 11 Floors) Fully Furnish...,24*7 Power Backup Lifts Swimming Pool +16 Amen...,"18.4566651,73.9095279",1509 Sq Feet,for,"2,000,000",2020-03-18,Building: Eternal Exotica Broker: Ozone Pro...
23,2 BHK Residential Apartment for Rent at Gyanga...,https://property.sulekha.com/apartments-flats-...,Wireless,1 Car Parking 1st (of 5 Floors) UnFurnished Fa...,Corporation water Drainage Facility Lifts +10 ...,"18.564194,73.8040924",900 Sq Feet,"15,000","50,000",2021-04-12,Building: Gyanganga Owner: Shrikant DALVI


In [133]:
df.to_csv(r"C:\Users\swaraj\Desktop\codes\modules\working\sulekha.csv")